# はじめに

本ノートブックでは **プロビジョニング汎用コンピューティング** をアタッチしてください。
学習観点でのわかりやすさのため、本ノートブックでは一部で hive_metastore や DBFS を利用しますが、これらの機能がサーバレスクラスタを通じて利用できないためです。

**注意**  
hive_metastore や DBFS はいずれもセキュリティ観点で利用が非推奨となっており、現在はそれぞれ後継機能（Unity Catalog、外部ローケーション）が提供されいます。そうした背景もあり、hive_metastore や DBFS はサーバレスコンピューティングではサポートされていません。  
よって本ノートブックではプロビジョニング汎用コンピューティングを使用しますが、hive_metastore や DBFS へのオペレーション自体は後継機能（Unity Catalog、外部ローケーション）でも同様であるためそれぞれ後継機能で置き換えることで動作します。

# 1. Spark API

## このデータはどのように表現されどのように操作するのか？

Spark では任意の言語（SQL、Python、Scala、R）を通じて任意の Spark データ構造（RDD、DataFrame、Dataset）に対して同じクエリを表現可能です。

![クエリ実行エンジン](https://files.training.databricks.com/images/aspwd/spark_sql_query_execution_engine.png)

Spark 初期バージョンではデータセットの低レベル表現である RDD を直接操作するコードを記述する必要がありました。

![Unified Engine](https://files.training.databricks.com/images/105/unified-engine.png)

しかし現在ではより高レベル（ユーザーフレンドリー）な API である DataFrame が主流でパフォーマンス改善もなされています。

![RDD vs DataFrames](https://files.training.databricks.com/images/105/rdd-vs-dataframes.png)

## データフレームの作成

実際にデータフレームを作成してみましょう。
ここでは例として CSV データをデータソースとしています。

In [0]:
covid_df = spark.read.csv("dbfs:/databricks-datasets/COVID/covid-19-data/us-counties.csv")

In [0]:
display(covid_df.limit(10))

In [0]:
covid_df.printSchema()

ヘッダ行がデータレコードになっていたり、データ型がすべて String になっていたりと期待した通りのデータフレームが作成できていないようです。

スキーマが明示されていない CSV をロードする際にはいくつかのオプションを指定することができます。

In [0]:
covid_df = spark.read.csv("dbfs:/databricks-datasets/COVID/covid-19-data/us-counties.csv", header=True, inferSchema=True)

In [0]:
display(covid_df.limit(10))

In [0]:
covid_df.printSchema()

レコード数はいくつでしょうか？

In [0]:
covid_df.count()

クエリをしてみましょう。

In [0]:
display(covid_df
 .sort(covid_df["date"].desc()) 
 .filter(covid_df["county"] == "Los Angeles") 
 .limit(10))

上記の Spark ジョブはどのようなものになるのでしょうか？  

</br><img src="../images/spark.1.png" width="600"/>  


## トランスフォーメーション と アクション

Spark の各種オペレーションは **トランスフォーメーション** と **アクション** に大きく分類されます。

それぞれ以下のように説明されます。
* トランスフォーメーションは **怠惰(LAZY)** です
* アクションは **懸命(EAGER)** です

どういうことでしょうか？

In [0]:
(covid_df
 .sort(covid_df["date"].desc()) 
 .filter(covid_df["county"] == "Los Angeles")) 

クエリの結果は表示されず実行は即座に完了します。それは **sort** と **filter** は遅延評価される`トランスフォーメーション`であるためです。

遅延評価にはいくつかのメリットがあります。
* 最初からすべてのデータをロードする必要がありません。
  * リソースにも限界がありますし非効率です。
* オペレーションの並列化が容易です。
  * 単一マシン、単一スレッド、単一のデータ要素に対して、N個の異なるトランスフォーメーションを処理することが可能です。
* 最も重要なことは遅延評価により様々な最適化を適用できるようになります。
  * 最適化を担う [Spark オプティマイザー（**Catalyst**）](https://www.databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html)ができることは様々です。
  
![Catalyst](https://files.training.databricks.com/images/105/catalyst-diagram.png)

In [0]:
(covid_df
 .sort(covid_df["date"].desc()) 
 .filter(covid_df["county"] == "Los Angeles") 
 .show()) 

結果が返ってきました。すなわち **show** は`アクション`であることがわかります。

最適化されたアクションのプランは Spark UI の Spark ジョブに関連づいたクエリーから確認できます。

</br><img src="../images/spark.2.png" width="600"/>  

## Spark SQL

Python ではなく SQL でクエリすることも可能です。

例として先ほどまで利用しているデータフレームをテーブルとして永続化しそれを SQL からクエリしてみましょう。

In [0]:
covid_df.createOrReplaceTempView("covid_table")

In [0]:
%sql
SELECT * FROM covid_table LIMIT 10;

以下は先ほど Python でクエリした内容を SQL で表現したものです。

In [0]:
%sql
SELECT * FROM covid_table WHERE county = "Los Angeles" ORDER BY date DESC;

Databricks visualization. Run in Databricks to view.

使い慣れた SQL で自由にクエリができます。

In [0]:
%sql
SELECT max(cases) AS max_cases, max(deaths) AS max_deaths, county 
FROM covid_table 
GROUP BY county 
ORDER BY max_cases DESC
LIMIT 10;

## Spark による簡単な分析

covid-19 データセットに加えて census.gov の国勢調査データを関連付けて簡単な分析をしてみましょう。

In [0]:
%sh wget https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv && cp co-est2019-alldata.csv /dbfs/tmp

In [0]:
census_df = spark.read.csv("dbfs:/tmp/co-est2019-alldata.csv", header=True, inferSchema=True)
display(census_df.limit(10))

covid-19 内の fipsコード列（連邦情報処理標準における州群コード）に対応する fipsコード列を国勢調査データ内で生成します。

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def make_fips(state_code, county_code):
  if len(str(county_code)) == 1:
    return str(state_code) + "00" + str(county_code)
  elif len(str(county_code)) == 2:
    return str(state_code) + "0" + str(county_code)
  else:
    return str(state_code) + str(county_code)

make_fips_udf = udf(make_fips, StringType())
  
census_df = census_df.withColumn("fips", make_fips_udf(census_df.STATE, census_df.COUNTY))

両データセットを FIPS コード で INNER JOIN します。

In [0]:
covid_with_census = (covid_df
                     .na.drop(subset=["fips"])
                     .join(census_df.drop("COUNTY", "STATE"), on=['fips'], how='inner'))

In [0]:
display(covid_with_census.limit(10))

人口の多い郡（人口200万人以上の群）での感染者数の推移を見てみましょう。

In [0]:
display(covid_with_census.filter("POPESTIMATE2019 > 2000000").select("county", "cases", "date"))

Databricks visualization. Run in Databricks to view.

covid-19 データセットは日毎に新しい行が追加されるので、感染者数は日毎に増加します。郡ごとの最新の数のみを取得しましょう。

In [0]:
from pyspark.sql.functions import row_number, col
from pyspark.sql import Window

w = Window.partitionBy("fips").orderBy(col("date").desc())
current_covid_rates = (covid_with_census
                       .withColumn("row_num", row_number().over(w))
                       .filter(col("row_num") == 1)
                       .drop("row_num"))

In [0]:
display(current_covid_rates.limit(1000))

最も困難に直面した郡はどこでしょうか？
ここでは総人口に対する感染者数の割合を見てみましょう。

In [0]:
current_covid_rates = (current_covid_rates
                       .withColumn("case_rates_percent", 100*(col("cases")/col("POPESTIMATE2019")))
                       .sort(col("case_rates_percent").desc()))

# トップ10の郡を参照します
display(current_covid_rates.select("county", "state", "cases", "POPESTIMATE2019", "case_rates_percent").limit(10))

# 2. Pandas と Pyspark によるデータ処理

## Pandasとは？

データ処理と言えば Pandas だ！

Pandas は構造化データとしてのデータの読み込みや読み込んだデータに対するデータ処理（加工）を行うために最も使用されている Python ライブラリの一つです。Pandas ライブラリは、データ分析、機械学習、データサイエンスプロジェクトなどで多く使われています。

Pandas は CSV、JSON、SQL などのフォーマットからデータをロードすることができ、SQL テーブルと同じように行と列を含む構造化オブジェクトであるデータフレームを作成します。

分散処理をサポートしていないので増大するデータをサポートするために追加の馬力を必要とした際はより強力なマシンを導入する必要があります。また Pandasデータフレームは遅延評価はされず可変です。

### Pandasデータフレームの例
PythonでPandasライブラリを使用するためには、import pandas as pd を用いてインポートします。

In [0]:
import pandas as pd    
data = [["James","","Smith",30,"M",60000], 
        ["Michael","Rose","",50,"M",70000], 
        ["Robert","","Williams",42,"",400000], 
        ["Maria","Anne","Jones",38,"F",500000], 
        ["Jen","Mary","Brown",45,None,0]] 
columns=['First Name','Middle Name','Last Name','Age','Gender','Salary']

# Create the pandas DataFrame 
pandasDF=pd.DataFrame(data=data, columns=columns) 
  
# print dataframe. 
print(pandasDF)


### Pandas データフレームの関数
Pandasデータフレームに実行できる関数のいくつかを以下に示します。デフォルトでそれぞれのカラムに対して関数が適用されます。

- df.count() – それぞれのカラムのカウントを返します(カウントには非null値のものだけが含まれます)。
- df.corr() – データフレーム内のカラムの相関を返します。
- df.head(n) – 上からn行を返します。
- df.max() – それぞれのカラムの最大値を返します。
- df.mean() – それぞれのカラムの平均値を返します。
- df.median() – それぞれのカラムの中央値を返します。
- df.min() – それぞれのカラムの最小値を返します。
- df.std() – それぞれのカラムの標準偏差を返します。
- df.tail(n) – 最後のn行を返します。

In [0]:
print("--- count() ---")
print(pandasDF.count())
print("\n--- max() ---")
print(pandasDF.max())
print("\n--- mean() ---")
print(pandasDF.mean())
print("\n---")

## PySparkとは？

シングルマシンで実行する Pandas に対して PySpark は複数マシンでの分散処理で実行します。より大きなデータセットを取り扱う場合には PySpark が最適です。

Python で Spark 処理を記述する PySpark はデータサイエンス、機械学習コミュニティで広く利用されています。

PySpark は Apache Spark の機能を用いて Python を実行するための Pythonで記述されたライブラリです。PySpark を用いることで分散クラスターでアプリケーションを並列に実行することができます。

言い換えると Apache Spark は大規模かつパワフルな分散データ処理、機械学習アプリケーションのための分析処理エンジンです。

![](https://qiita-user-contents.imgix.net/https%3A%2F%2Fi1.wp.com%2Fsparkbyexamples.com%2Fwp-content%2Fuploads%2F2020%2F08%2FWhat-is-PySpark.png%3Fresize%3D1024%252C164%26ssl%3D1?ixlib=rb-4.0.0&auto=format&gif-q=60&q=75&s=ce59994dc84b42a2caddb1d81a7f797c)

Spark は基本的に Scala で記述されており、以降業界での導入が進んだことで Py4J を用いて Python 向け API PySpark がリースされました。Py4J は PySpark 内でインテグレーションされており Python が動的に JVM オブジェクトとやりとりすることを可能にしているので PySpark を実行するには Python、Apache Spark と Java をインストールする必要があります。さらに、開発においては PySpark アプリケーションを実行するために Spyder IDE や Jupyter notebook のような有用なツールを数多く備えている(機械学習コミュニティで広く使われている) Anaconda ディストリビューションを使うこともできます。

Spark 環境をマネージドサービスとして提供する Databricks はおいてはこうしたセットアップを意識する必要はありません。

### PySpark の機能
- インメモリの計算処理
- 並列化による分散処理
- さまざまなクラスターマネージャ(Spark、Yarn、Mesosなど)で利用可能
- フォールトトレラント
- イミュータブル
- 遅延評価
- キャッシュ & 永続化
- データフレームを使う際のビルトインの最適化処理
- ANSI SQLのサポート

### PySpark の特徴
- PySparkは、分散によってデータを効率的に処理できる汎用、インメモリ、分散処理エンジンです。
- データ取り込みパイプラインにおいてPySparkを用いることで非常に大きな効果を得ることができます。
- PySparkを用いることで、Hadoop HDFS、Azure Storage、AWS S3、Google GCSなど数多くのファイルシステムのデータを処理することができます。
- PySparkはストリーミングやKafkaを用いてリアルタイムデータを処理することに使うことができます。
- PySparkのストリーミングを用いることで、ファイルシステムやソケットからのストリームからファイルをストリーミングすることができます。
- PySparkはネイティブで機械学習、グラフライブラリを持っています。

### PySpark 優位性

- お使いのデータが膨大かつ毎年増加し続けており、処理時間を改善したい。
- フォールトトレラントが必要（分散処理による耐性強化）。
- ANSI SQL互換性や使用する言語のバライエティ(Python、Scala、Java、R)。
- 外部データとの接続性（Parquet、Avro、Hive、Casandra、Snowflakeなどからデータを読み込みたい）。
- データをストリーミングしリアルタイムで処理したい。

**従来、分散コンピューティングである Spark 環境のセットアップや運用は大変でしたが、Spark をマネージドサービスとして提供する Databricks はおいてはこうしたセットアップた運用を意識する必要が無くなります。**

### PySparkデータフレームの例
先ほどの Pandas での記述に相当する PySpark コードがこちらです。

In [0]:
data = [("James","","Smith",30,"M",60000),
        ("Michael","Rose","",50,"M",70000),
        ("Robert","","Williams",42,"",400000),
        ("Maria","Anne","Jones",38,"F",500000),
        ("Jen","Mary","Brown",45,"F",0)]

columns = ["first_name","middle_name","last_name","Age","gender","salary"]
pysparkDF = spark.createDataFrame(data = data, schema = columns)
pysparkDF.printSchema()
pysparkDF.show(truncate=False)

In [0]:
from pyspark.sql.functions import mean, col, max
#Example 1
df2=pysparkDF.select(mean("age"),mean("salary")) \
             .show()
#Example 2
pysparkDF.groupBy("gender") \
         .agg(mean("age"),mean("salary"),max("salary")) \
         .show()


## Pandas API on Spark

とはいえ、データ処理と言えば Pandas だ！そんなエンジニアのために Pandas API を利用しつつ Spark 処理できる API として **Pandas API on Spark**(旧 Koalas) があります。

pandas_api() によって Spark データフレームを Pandas API でデータ処理できるようになります。

In [0]:
import pyspark.pandas as ps
psdf = current_covid_rates.pandas_api()

pandas_api() による変換後は Pandas 作法でデータ処理を記述できます。

In [0]:
psdf['state']

In [0]:
psdf.head(2)

In [0]:
psdf.describe()

In [0]:
psdf.sort_values(by='deaths', ascending=False).head(10)

## データフレームの相互変換

もしくは、それぞれのデータフレームを相互変換することも可能です。

Pandas データフレーム から Spark データフレーム の作成

In [0]:
# Create PySpark DataFrame from Pandas
pysparkDF = spark.createDataFrame(pandasDF) 
pysparkDF.printSchema()
pysparkDF.show()


PySpark データフレーム から Pandas データフレーム の作成

**留意**：toPandas()メソッドは、データをSparkドライバーのメモリーに集めるアクションなので、大規模データセットを取り扱っている際には注意を払う必要があります。収集したデータがSparkドライバーのメモリーに収まらない際にはOutOfMemoryExceptionに遭遇することになります。

In [0]:
#Convert PySpark to Pandas
pandasDF = pysparkDF.toPandas()
print(pandasDF)


# 参考
- [今さら聞けないPython - Sparkのご紹介](https://qiita.com/taka_yayoi/items/5415db284b96c7c60d44)
- [Pandas vs PySpark DataFrame With Examples](https://sparkbyexamples.com/pyspark/pandas-vs-pyspark-dataframe-with-examples/amp/)
- [Spark 上の Pandas API](https://learn.microsoft.com/ja-jp/azure/databricks/pandas/pandas-on-spark)]
- [PySpark と pandas DataFrame 間で変換する](https://learn.microsoft.com/ja-jp/azure/databricks/pandas/pyspark-pandas-conversion)
- [pandasユーザーがPandas API on Sparkでつまづいたあれこれ](https://kakehashi-dev.hatenablog.com/entry/2022/12/24/090000)
- [Ray on Azure Databricks とは](https://learn.microsoft.com/ja-jp/azure/databricks/machine-learning/ray/)